<code>plot_RMSE_MAE_S2S_models_VRILE_counts_LINE_PLOTS.ipynb</code>.  This notebook plots VRILE count as a function of region, month, and lead time. 

<b>inputs:</b><br>
<li>  model name (ecmwf,ukmo,ncep,metreofr) </li>
<li>  seas_str [string for season; ALL if we want to do full year]</li>
<li>  seas_sel [months of season; empty if we want to do full year] </li>
<li>  vrile_thresh [threshhold at which VRILE is estimated </li>
<li>  thresh_str [string for VRILE threshhold] </li>


In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import seaborn as sns

## Functions

<li>1) Load model netCDF files, combine with CTRL, and use common reforecast period. <br>
if NCEP, use entire period </li>
<li> 2) Add aggregate regions </li>
<li> 3) Create climatology--model: calculate date of year for valid date, lead time in weeks.<br>
    Group by region, lead time, and valid date of year <br>
    Average climatology based on day of year and lead time in weeks--use <code>transform</code> to create <code>SIE_clim</code>.<br> 
    Subtract <code>SIE_clim</code> from <code>SIE</code><br> </li>
<li> 4) Create observed climatology </li>
<li> 5) Calculate VRILE threshold based on <code>seas_sel</code> </li>
<li> 6) Count number of VRILE days as a function of region, month, and lead week </li>

Load the netCDF files for our specific model from <code>/home/disk/sipn/nicway</code>.  We want the control runs too.  Then, select only the common reforecast period (1999-01-01 to 2014-12-31), and add the control run to the rest of the output. <br>
NOTE: for NCEP, since the reforecast period is short (ends in 2011), we will just use the entire period. 

In [2]:
def load_model(model_name):
    # Paths for perturb and control runs
    filepath = '/home/disk/sipn/nicway/data/model/{model_name}/reforecast/sipn_nc_agg/'.format(model_name=model_name)
    filepath_ctrl = '/home/disk/sipn/nicway/data/model/{model_name}/reforecast.control/sipn_nc_agg/'.format(model_name=model_name)
    # Open both with xarray
    filenames = xr.open_mfdataset(filepath+'/*.nc',combine='by_coords')
    filenames_ctrl = xr.open_mfdataset(filepath_ctrl+'/*.nc',combine='by_coords')
    print(filenames)
    # load SIE
    SIE = filenames.Extent
    SIE_ctrl = filenames_ctrl.Extent
    # Add coordinate to ensemble dimension for SIE_ctrl so we can combine with SIE
    SIE_ctrl.coords['ensemble'] = xr.DataArray([len(SIE.ensemble)],
                                               dims='ensemble', coords={'ensemble':[len(SIE.ensemble)]})
    # Use common reforecast period for all models EXCEPT NCEP
    if model_name == 'ncep':
        # Don't need to use a common reforecast but remove repeated indices
        _,init_ind = np.unique(SIE['init_time'],return_index=True)
        _,init_ind_c = np.unique(SIE_ctrl['init_time'],return_index=True)
        SIE = SIE.isel(init_time=init_ind)
        SIE_ctrl = SIE_ctrl.isel(init_time=init_ind_c)
    elif model_name != 'ncep':
        common_start = '1999-01-01'
        common_end = '2014-12-31'
        # Select only common reforecast period (full period for NCEP)
        SIE = SIE.sel(init_time=slice(common_start,common_end))
        SIE_ctrl = SIE_ctrl.sel(init_time=slice(common_start,common_end))
        # Remove repeated indices in CTRL
        _,init_ind_c = np.unique(SIE_ctrl['init_time'],return_index=True)
        SIE_ctrl = SIE_ctrl.isel(init_time=init_ind_c)
    # Concatenate the two
    SIE = xr.concat([SIE,SIE_ctrl],dim='ensemble')
    
    return SIE

We want to create a few aggregate regions from the NSIDC MASIE regions (more at: https://nsidc.org/data/masie/browse_regions) since some regions are so small. We're going to combine the following: <br>
<li> Kara and Laptev Seas (<code>region_KL</code>)</li>
<li> Barents, Kara and Laptev Seas (<code>region_BKL</code>)</li>
<li> East Siberian, Beaufort, and Chukchi Seas (<code>region_EBC</code>)</li>
<li> Atlantic (Baffin Bay and East Greenland Sea) (<code>region_ATL</code>)</li>
<li> East Siberian, Beaufort, Chukchi, Laptev Seas (<code>region_EBCL</code>)</li>


In [3]:
def create_aggregate_regions(SIE_data):
    nregions = SIE_data['nregions']
    region_names = SIE_data['region_names']
    # Get corresponding indices for each of our aggregate regions
    region_KL = nregions[region_names.isin(['Kara Sea','Laptev Sea'])]
    region_BKL = nregions[region_names.isin(['Barents Sea','Kara Sea','Laptev Sea'])]
    region_EBC = nregions[region_names.isin(['East Siberian Sea','Beaufort Sea','Chukchi Sea'])]
    region_ATL = nregions[region_names.isin(['Baffin Bay','East Greenland Sea'])]
    region_EBCL = nregions[region_names.isin(['East Siberian Sea','Beaufort Sea','Chukchi Sea','Laptev Sea'])]
    # Select each aggregate region, add them together, and add the 'nregions' dimension back; concatenate all aggregates 
    SIE_agg = xr.concat([SIE_data.sel(nregions=region_KL).sum(dim='nregions').expand_dims(dim='nregions'),
                  SIE_data.sel(nregions=region_BKL).sum(dim='nregions').expand_dims(dim='nregions'),
                  SIE_data.sel(nregions=region_EBC).sum(dim='nregions').expand_dims(dim='nregions'),
                  SIE_data.sel(nregions=region_ATL).sum(dim='nregions').expand_dims(dim='nregions'),
                  SIE_data.sel(nregions=region_EBCL).sum(dim='nregions').expand_dims(dim='nregions')],dim='nregions')
    # Add coordinates to nregions.  Start at 20 to make a clear separation from original NSIDC regions
    SIE_agg = SIE_agg.assign_coords(nregions=[20,21,22,23,24])
    # Add region names
    region_names_extra = ['Kara-Laptev Sea','Barents-Kara-Laptev Sea','East Siberian-Beaufort-Chukchi Sea',
                      'Atlantic','East Siberian-Beaufort-Chukchi-Laptev Sea']
    SIE_agg["region_names"] = ("nregions",region_names_extra)
    #SIE
    SIE_data = xr.concat([SIE_data,SIE_agg],dim='nregions')
    return(SIE_data)

Create climatology for model output based on region, day of year, and lead time.  Get month-day for valid dates (don't use dayofyear because of leap days).  Since our forecasts are not initialized every day, we will do two versions--one where we keep each lead time separate, and one where we group our lead time climatology based on week instead of day (which is more supported in the literature). 

In [4]:
def create_model_climatology(SIE,week_length):
    # Add valid date in %m-%d format
    SIE['valid date of yr'] = pd.to_datetime(SIE['valid date']).dt.strftime('%m-%d')
    # Determine lead time as a function of weeks instead of days
    SIE_df_weekly = SIE.copy()
    SIE_df_weekly['lead time (weeks)'] = SIE_df['lead time (days)'].values.astype('timedelta64[D]')/pd.Timedelta(week_length,'D')
    SIE_df_weekly['lead time (weeks)'] = SIE_df_weekly['lead time (weeks)'].apply(np.floor)
    # Group by region, lead time, and valid day of year. Use .transform('mean') so that our climatology has the 
    # same shape as the original input dataframe (so we can just subtract SIE_clim from SIE easily at the end)
    SIE['SIE clim'] = SIE.groupby(['region','lead time (days)','valid date of yr'])['SIE'].transform('mean')
    SIE_df_weekly['SIE clim'] = SIE_df_weekly.groupby(['region','lead time (weeks)','valid date of yr'])['SIE'].transform('mean')

    SIE['SIE anom'] = SIE['SIE'] - SIE['SIE clim']
    SIE_df_weekly['SIE anom'] = SIE_df_weekly['SIE'] - SIE_df_weekly['SIE clim']
    return(SIE,SIE_df_weekly)

Create climatology for the observations.  This is easier, we just need to get the day of year (month-day) for each observation, take the mean, and then subtract the annual cycle from our full dataset. 

In [5]:
def create_obs_climatology(SIE):
    # Add valid date in %m-%d format
    SIE['valid day of year'] = pd.to_datetime(SIE['valid date']).dt.strftime('%m-%d')
    # Group by region and day of year and take the mean. Use transform to make the output match the dataframe instead
    # of creating a multi-index
    SIE['SIE clim'] = SIE.groupby(['region','valid day of year'])['SIE'].transform('mean')
    # And simply subtract SIE_clim from actual SIE
    SIE['SIE anom'] = SIE['SIE'] - SIE['SIE clim']
    return SIE

Calculate VRILE days from obs.  Determine $n$-day changes in SIE and anomalous SIE, and estimate the lower tail of the distribution based on <code>VRILE_thresh</code>.  For now, we'll assume we need a 5-day change in SIE.  

In [6]:
def calc_VRILE_thresh(obs_SIE,plot_quant,vrile_thresh,nday_change,seas_sel,seas_str):
    # Use shift to move SIE/SIE anom foreward and backward by nday_change days
    nday_shift = np.floor(nday_change/2)
    obs_SIE['SIE anom -n'] = obs_SIE.groupby(['region'])['SIE anom'].shift(-nday_shift)
    obs_SIE['SIE anom +n'] = obs_SIE.groupby(['region'])['SIE anom'].shift(+nday_shift)
    obs_SIE['SIE -n'] = obs_SIE.groupby(['region'])['SIE'].shift(-nday_shift)
    obs_SIE['SIE +n'] = obs_SIE.groupby(['region'])['SIE'].shift(+nday_shift)
    obs_SIE['d_SIE'] = obs_SIE['SIE -n'] - obs_SIE['SIE +n']
    obs_SIE['d_SIE anom'] = obs_SIE['SIE anom -n'] - obs_SIE['SIE anom +n']
    # 
    region_names = obs_SIE['region'].unique().tolist()
    obs_SIE = obs_SIE.set_index('region')
    # Trim to correct season based on seas_sel
    obs_SIE_VRILE_only = pd.DataFrame()
    for ireg in region_names:
        SIE_ireg = obs_SIE.loc[ireg]
        if seas_str == 'ALL':
            SIE_ireg['p05'] = SIE_ireg[plot_quant].quantile(vrile_thresh)
        else:
            SIE_ireg['p05'] = SIE_ireg[SIE_ireg['valid date month'].isin(seas_sel)][plot_quant].quantile(vrile_thresh)
        SIE_ivrile = SIE_ireg.where(SIE_ireg[plot_quant]<=SIE_ireg['p05']).dropna(how='all')
        obs_SIE_VRILE_only = obs_SIE_VRILE_only.append(SIE_ivrile)

    return obs_SIE_VRILE_only

Calculate raw error, mean absolute error, and RMSE.  These calculations will be a function of region and lead time.  See nice notebook for equations for RMSE and MAE

In [7]:
def calculate_errors(model_SIE,obs_SIE):
    model_SIE['lead days'] = model_SIE['lead time (days)'].dt.days
    # Group by region, valid date, and lead time (for model)
    SIE_obsx = obs_SIE.groupby(['region','valid date'])['SIE','SIE clim','SIE anom'].mean()
    # First calculate raw model error (we'll also group by init time so we can save that for calculating other stuff)
    SIE_model_raw = model_SIE.groupby(['region','valid date','lead days','init date'])['SIE','SIE clim','SIE anom'].mean()
    SIE_raw_error = SIE_model_raw[['SIE','SIE anom']] - SIE_obsx[['SIE','SIE anom']]
    SIE_raw_error = SIE_raw_error.dropna(how='all')
    # Now, we'll do MAE and RMSE (we don't care about the date of initialization)
    SIE_modelx = model_SIE.groupby(['region','valid date','lead days'])['SIE','SIE clim','SIE anom'].mean()
    SIE_diff = SIE_modelx[['SIE','SIE anom']] - SIE_obsx[['SIE','SIE anom']]
    # Square errors to get RMSE. get absolute value of errors for MAE
    SIE_diff = SIE_diff.dropna(how='all')
    SIE_diff = SIE_diff.rename(columns={'SIE':'SIE raw error','SIE anom':'SIE anom raw error'})
    SIE_diff[['SIE sq error','SIE anom sq error']] = SIE_diff**2
    # Now, average over all valid dates and take the square root. 
    SIE_errors = SIE_diff[['SIE sq error','SIE anom sq error']].mean(level=(0,2))**0.5
    # Absolute value, then average over all valid dates to get the MAE
    SIE_errors[['SIE MAE','SIE anom MAE']] = SIE_diff[['SIE raw error','SIE anom raw error']].abs().mean(level=(0,2))
    SIE_errors = SIE_errors.rename(columns={'SIE sq error':'SIE RMSE','SIE anom sq error':'SIE anom RMSE'})
    return(SIE_raw_error,SIE_errors)
    

In [9]:
model_name = 'ecmwf'
seas_str = 'ALL'
seas_sel = []
obs_name = 'NSIDC_0079'
WEEKLY = False
vrile_thresh = 0.05
thresh_str = '05'
nday_change = 5 #number of days for VRILE calculation
lead_day_min = 0
lead_day_max = 7
COMMON_REFORECAST = True

Load model output for our desired model

In [10]:
SIE = load_model(model_name)
print('loaded ',model_name)

<xarray.Dataset>
Dimensions:       (ensemble: 10, fore_time: 46, init_time: 2080, nregions: 15)
Coordinates:
    region_names  (nregions) object dask.array<chunksize=(15,), meta=np.ndarray>
  * fore_time     (fore_time) timedelta64[ns] 0 days 1 days ... 44 days 45 days
  * ensemble      (ensemble) int32 0 1 2 3 4 5 6 7 8 9
  * nregions      (nregions) int64 99 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * init_time     (init_time) datetime64[ns] 1998-08-06 ... 2018-08-01
Data variables:
    Extent        (ensemble, init_time, fore_time, nregions) float64 dask.array<chunksize=(10, 1, 46, 15), meta=np.ndarray>
loaded  ecmwf


Create aggregate regions that combine some of the NSIDC-MASIE regions

In [11]:
SIE = create_aggregate_regions(SIE)
print('combined regions')

combined regions


Now, take the ensemble mean and get lead time in days

In [12]:
SIE_ens_mean = SIE.mean(dim='ensemble')
regions = SIE.region_names
lead_days = SIE.fore_time.dt.days

Convert to dataframe because I like Pandas

In [13]:
SIE_df = SIE_ens_mean.to_dataframe().reset_index()

Calculate the date for forecasts by adding the <code>fore_time</code> to <code>init_time</code>. Rename some columns to make life easier

In [14]:
SIE_df['valid date'] = SIE_df['init_time'] + SIE_df['fore_time']
SIE_df = SIE_df.rename(columns={'region_names':'region',
                           'fore_time':'lead time (days)',
                           'init_time':'init date',
                           'Extent':'SIE'})

If we want only the common reforecast period (<code>COMMON_REFORECAST == True</code>), trim now to 1999-2014 except for <code>NCEP</code>. 

In [15]:
if (COMMON_REFORECAST == True) & (model_name != 'ncep'):
    SIE_df = SIE_df[pd.to_datetime(SIE_df['init date']).dt.year.isin(np.arange(1999,2015))]

Create climatology for model output.  Decide how long we want weeks to be for weekly climatology (default is 7 days)

In [16]:
week_length = 7
SIE_df,SIE_df_weekly = create_model_climatology(SIE_df,7)
print('model climatology created')

model climatology created


Load observations.  NSIDC_0079 is NASA Bootstrap, NSIDC_0081 is NASA team

In [17]:
obs_type = 'sipn_nc_yearly_agg'
filepath = '/home/disk/sipn/nicway/data/obs/{model_name}/{model_type}/'.format(model_name=obs_name,
                                                                              model_type=obs_type)
obs_filenames = xr.open_mfdataset(filepath+'/*.nc',combine='by_coords')
print('opening ',obs_filenames)
obs_SIE = obs_filenames.Extent
obs_regions = obs_filenames.nregions
obs_region_names = obs_filenames['region_names'].values
# Drop region names and re-add as a non-dask.array object.  This is stupid but oh well
obs_SIE = obs_SIE.drop('region_names')
obs_SIE["region_names"] = ("nregions",obs_region_names)
print('obs loaded')

opening  <xarray.Dataset>
Dimensions:       (nregions: 15, time: 11322)
Coordinates:
    region_names  (nregions) object dask.array<chunksize=(15,), meta=np.ndarray>
  * nregions      (nregions) int64 99 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * time          (time) datetime64[ns] 1989-01-01 1989-01-02 ... 2019-12-31
Data variables:
    Extent        (time, nregions) float64 dask.array<chunksize=(365, 15), meta=np.ndarray>
obs loaded


Add aggregate regions to obs and convert obs to Pandas dataframe

In [18]:
obs_SIE = create_aggregate_regions(obs_SIE)
obs_SIE = obs_SIE.to_dataframe().reset_index()
obs_SIE = obs_SIE.rename(columns={'Extent':'SIE','region_names':'region','time':'valid date'})

Again trim to common reforecast if desired

In [19]:
if COMMON_REFORECAST == True:
    obs_SIE = obs_SIE[pd.to_datetime(obs_SIE['valid date']).dt.year.isin(np.arange(1999,2015))]

Calculate our observed climatology 

In [20]:
obs_SIE = create_obs_climatology(obs_SIE)
print('observed climatology created')
obs_SIE['valid date month'] = pd.to_datetime(obs_SIE['valid date']).dt.month

observed climatology created


In [22]:
obspath_save = '/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/data/OBS_{obs_name}_all_regions_climatology_created'.format(obs_name=obs_name)
if COMMON_REFORECAST == True:
    obspath_save = obspath_save+'_COMMON_REFORECAST.csv'
else:
    obspath_save = obspath_save+'_ALL_YEARS.csv'
obs_SIE.to_csv(obspath_save)

Get observed 

In [20]:
obs_SIE_VRILE_only = calc_VRILE_thresh(obs_SIE,'d_SIE',vrile_thresh,nday_change,seas_sel,seas_str)
obs_SIE_anom_VRILE_only = calc_VRILE_thresh(obs_SIE,'d_SIE anom',vrile_thresh,nday_change,seas_sel,seas_str)

/home/disk/sipn/mcmcgraw/anaconda3/envs/sea_ice_variability_S2S/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now get model VRILE days as a function of lead week

In [21]:
SIE_df_weekly['valid date month'] = pd.to_datetime(SIE_df_weekly['valid date']).dt.month

In [22]:
SIE_df_weekly_gb = SIE_df_weekly.groupby(['lead time (weeks)','region',
                       'valid date','init date'])['SIE','SIE clim','SIE anom','valid date month'].mean()
SIE_model_VRILE = pd.DataFrame()
SIE_anom_model_VRILE = pd.DataFrame()
weeks_list = SIE_df_weekly_gb.index.get_level_values('lead time (weeks)').unique().tolist()
for iweek in weeks_list:
    i_SIE = calc_VRILE_thresh(SIE_df_weekly_gb.xs(iweek).reset_index(),'d_SIE',
                                         vrile_thresh,nday_change,seas_sel,seas_str)
    i_SIE['lead weeks'] = iweek
    i_SIE_anom = calc_VRILE_thresh(SIE_df_weekly_gb.xs(iweek).reset_index(),'d_SIE anom',
                                         vrile_thresh,nday_change,seas_sel,seas_str)
    i_SIE_anom['lead weeks'] = iweek
    SIE_model_VRILE = SIE_model_VRILE.append(i_SIE)
    SIE_anom_model_VRILE = SIE_anom_model_VRILE.append(i_SIE)


/home/disk/sipn/mcmcgraw/anaconda3/envs/sea_ice_variability_S2S/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/home/disk/sipn/mcmcgraw/anaconda3/envs/sea_ice_variability_S2S/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Save files!

In [23]:
clim_freq_str = 'WEEKLY'
fdir = '/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/data/{model_name}/'.format(model_name=model_name)
if COMMON_REFORECAST == True:
    fdir = fdir+'COMMON_REFORECAST/'
    
if not os.path.exists(fdir):
    os.makedirs(fdir)
fname_save_raw = fdir+'VRILE_count_SIE_{model_name}_months{seas_str}_VRILE{thresh_str}_model_clim_freq_{clim_freq_str}.csv'.format(model_name=model_name,
                                             seas_str=seas_str,thresh_str=thresh_str,clim_freq_str=clim_freq_str)
fname_save_RMSE = fdir+'VRILE_count_SIE_anom_{model_name}_months{seas_str}_VRILE{thresh_str}_model_clim_freq_{clim_freq_str}.csv'.format(model_name=model_name,
                                             seas_str=seas_str,thresh_str=thresh_str,clim_freq_str=clim_freq_str)
fname_save_obs = fdir+'VRILE_count_SIE_obs_months{seas_str}_VRILE{thresh_str}_model_clim_freq_{clim_freq_str}.csv'.format(
                                             seas_str=seas_str,thresh_str=thresh_str,clim_freq_str=clim_freq_str)
fname_save_obs_anom = fdir+'VRILE_count_SIE_anom_obs_months{seas_str}_VRILE{thresh_str}_model_clim_freq_{clim_freq_str}.csv'.format(
                                             seas_str=seas_str,thresh_str=thresh_str,clim_freq_str=clim_freq_str)
SIE_model_VRILE.to_csv(fname_save_raw)
SIE_anom_model_VRILE.to_csv(fname_save_RMSE)
#
obs_SIE_VRILE_only.to_csv(fname_save_obs)
obs_SIE_anom_VRILE_only.to_csv(fname_save_obs_anom)
print('files saved')

files saved


In [24]:
fname_save_raw

'/home/disk/sipn/mcmcgraw/McGraw_etal_2020/code/make_it_nice/data/ecmwf/COMMON_REFORECAST/VRILE_count_SIE_ecmwf_monthsALL_VRILE05_model_clim_freq_WEEKLY.csv'